In [1]:
# **************************
# Author: Sebastien Vezina
# Date: 05-Nov-2020
# **************************
import os
import requests
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
from functools import partial

from alpha_vantage.foreignexchange import ForeignExchange

# Install package before importing: pip install alpha-vantage
# https://pypi.org/project/alpha-vantage/

In [2]:
# Load .env enviroment variables
load_dotenv()
alpha_vantage_key = os.getenv("ALPHAVANTAGE")

In [3]:
fx = ForeignExchange(key=alpha_vantage_key,output_format='pandas')
cad_data, meta_data = fx.get_currency_exchange_daily(from_symbol='CAD', to_symbol='USD', outputsize='full')

cad_data.head()

,1. open,2. high,3. low,4. close
date,,,,
2020-11-06,0.7641,0.7679,0.7631,0.7655
2020-11-05,0.7607,0.7674,0.7584,0.7660
2020-11-04,0.7596,0.7634,0.7515,0.7609
2020-11-03,0.7562,0.7629,0.7551,0.7601
2020-11-02,0.7485,0.7565,0.7474,0.7561


In [4]:
# Keep only Date and Close columns
cad_data.drop(['1. open', '2. high', '3. low'], axis=1, inplace=True)

# Rename columns
cad_data.reset_index(inplace=True)
cad_data.columns = ['Date', 'Close']

cad_data.head()

,Date,Close
0,2020-11-06,0.7655
1,2020-11-05,0.7660
2,2020-11-04,0.7609
3,2020-11-03,0.7601
4,2020-11-02,0.7561


In [5]:
# Select only from 2015-10-01 to 2020-11-01
start_date = "01/11/2015"
end_date = "01/11/2020"

f = partial(pd.to_datetime, dayfirst=True)
cad_data = cad_data[(cad_data['Date'] > f(start_date)) & (cad_data['Date'] < f(end_date))]



# Add extra Year column
cad_data['Year'] = pd.DatetimeIndex(cad_data['Date']).year

# Change column order 
cad_data = cad_data[['Date', 'Year', 'Close']]

# Set index back
cad_data.set_index("Date", inplace = True)

# Save to CSV file
cad_data.to_csv('../../02-Data/Clean Data/Market Data/CAD_5yrs.csv')

cad_data

,Year,Close
Date,,
2020-10-30,2020,0.7499
2020-10-29,2020,0.7501
2020-10-28,2020,0.7503
2020-10-27,2020,0.7581
2020-10-26,2020,0.7565
...,...,...
2015-11-06,2015,0.7515
2015-11-05,2015,0.7591
2015-11-04,2015,0.7601


In [6]:
# GSPTSE from 2015-10-01 to 2020-11-01
SP_TSX500_file = Path("../../02-Data/Dirty Data/Market Data/SPTSX60 Historical.csv")
SP_TSX500_data = pd.read_csv(SP_TSX500_file, index_col="Date")
SP_TSX500_data

,Price,Open,High,Low,Vol.,Change %
Date,,,,,,
"Oct 30, 2020",928.90,930.88,932.10,919.76,112.89M,-0.63%
"Oct 29, 2020",934.83,934.78,939.48,926.36,101.86M,0.42%
"Oct 28, 2020",930.90,944.45,944.45,930.23,147.62M,-2.50%
"Oct 27, 2020",954.81,959.93,960.71,953.49,107.18M,-0.53%
"Oct 26, 2020",959.91,966.43,967.16,954.08,115.59M,-1.30%
...,...,...,...,...,...,...
"Nov 06, 2015",795.18,795.54,797.53,790.36,99.77M,0.03%
"Nov 05, 2015",794.97,801.74,803.09,793.64,114.20M,-0.99%
"Nov 04, 2015",802.92,806.91,810.12,801.62,117.72M,-0.25%


In [7]:
# Remove useles column
SP_TSX500_data.reset_index(inplace=True)

# Convert Date column to datetime type
SP_TSX500_data['Date'] = pd.to_datetime(SP_TSX500_data['Date'])

# Add extra Year column
SP_TSX500_data['Year'] = pd.DatetimeIndex(SP_TSX500_data['Date']).year
#GSPTSE_data['year'] = GSPTSE_data.loc[0: len(GSPTSE_data), 'date'].str.slice(0, 4)

# Change column order and keep only relevant ones
SP_TSX500_data_sliced = SP_TSX500_data[['Date', 'Year', 'Price']].set_index('Date')

SP_TSX500_data_sliced

,Year,Price
Date,,
2020-10-30,2020,928.90
2020-10-29,2020,934.83
2020-10-28,2020,930.90
2020-10-27,2020,954.81
2020-10-26,2020,959.91
...,...,...
2015-11-06,2015,795.18
2015-11-05,2015,794.97
2015-11-04,2015,802.92


In [8]:
# Save to CSV file
cad_data.to_csv('../../02-Data/Clean Data/Market Data/SP_TSX60_5yrs.csv')